<a href="https://colab.research.google.com/github/hadi1421/FYP/blob/main/SW%20with%20Naive%20bayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
import pandas as pd
import nltk
import numpy as np
from nltk.corpus import stopwords
import re
from nltk.stem import SnowballStemmer


In [4]:
from google.colab import files
v=files.upload()

Saving training.csv to training.csv


In [3]:
import pandas as pd
tweets=pd.read_csv('training.csv',encoding='latin',
                    names = ['sentiment','id','date','query','user','tweet'])

In [4]:
print("Dataset length",len(tweets))

Dataset length 1600000


In [5]:
#removing uncessary columns
tweets.drop(columns=['date','query','user'], inplace=True)
tweets=tweets[['tweet', 'sentiment']]

In [6]:
tweets.isna().sum()

tweet        0
sentiment    0
dtype: int64

In [7]:
tweets['tweet'] =tweets['tweet'].replace(4,1)

In [45]:
stemmer = nltk.SnowballStemmer("english")
STOPWORDS = stopwords.words('english')
STOPWORDS.remove('not')

#function to preprocess text by converting to lower case, removing mentions and links and stem the words and remove stopwords
def clean_text(tweet):
    
    tweet = str(tweet).lower()                                           # converting to lower case
    tweet = re.sub(r'@\S+|http\S+|www.\S+|\n','',tweet)                  # removing mentions and links
    tweet = re.sub(r'[^A-Za-z0-9\s]+', '', tweet)                        # removing special characters 
    
    tweet = [stemmer.stem(word) for word in tweet.split(' ')]            # stem words 
    tweet = " ".join([word for word in tweet if word not in STOPWORDS])  # remove stopwords
    tweet = tweet.strip()                                                # remove extra spaces from start and end of string
     
    return tweet


tweets['tweet'] = tweets['tweet'].apply(clean_text)


In [46]:
tweets.head()

,tweet,sentiment
0,awww bummer shoulda got david carr third day,0
1,upset cant updat facebook text might cri resul...,0
2,dive mani time ball manag save 50 rest go bound,0
3,whole bodi feel itchi like fire,0
4,not behav im mad whi becaus cant see,0


In [47]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline

#creating pipeline to vectorize text and fit in the model
model = make_pipeline(TfidfVectorizer(ngram_range=(1,2)), MultinomialNB(alpha=1.0))

In [48]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train,y_test = train_test_split(tweets['tweet'],tweets['sentiment'],test_size =0.2 , random_state = 42)

In [49]:
model.fit(X_train,y_train)

Pipeline(memory=None,
         steps=[('tfidfvectorizer',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 2), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('multinomialnb',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

In [50]:
predicted = model.predict(X_test)

In [51]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,predicted)

0.785534375